In [1]:
%load_ext autoreload

In [11]:
%autoreload 2
import hydra
import sys
import torch

import os
import hydra
import torch
import wandb
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, Callback

from RegularizingEmbeddings.lightning.train import MetricsCallback
from RegularizingEmbeddings.lightning.model import SequenceModel
from RegularizingEmbeddings.lightning.data import SequenceDataModule
from RegularizingEmbeddings.data.data_generation import make_trajectories, postprocess_data, generate_train_and_test_sets


/nfs/home2/eisenaj/code/RegularizingEmbeddings/RegularizingEmbeddings/lightning/train.py:37: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1


In [44]:
# load the config
with hydra.initialize(version_base="1.3", config_path="../conf"):
    config = hydra.compose(config_name="config")


In [46]:
"""Main training routine.

Args:
    config: Hydra configuration
"""
# Set up W&B logging
wandb_logger = WandbLogger(
    project=config.training.wandb.project,
    name=config.training.wandb.name,
    config=OmegaConf.to_container(config, resolve=True),
)

# Initialize callbacks
callbacks = [
    ModelCheckpoint(
        dirpath=os.path.join(config.training.paths.output_dir, "checkpoints"),
        filename="model-{epoch:02d}-{val_loss:.4f}",
        monitor="val_loss",
        mode="min",
        save_top_k=1,
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=config.training.training.patience,
        mode="min",
    ),
    MetricsCallback(),
]

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=config.training.training.max_epochs,
    accelerator='auto',
    devices='auto',
    logger=wandb_logger,
    callbacks=callbacks,
)

# Initialize data module
torch.random.manual_seed(config.data.flow.random_state)
eq, sol, dt = make_trajectories(config)
values = postprocess_data(config, sol)
# Postprocess data
values = postprocess_data(config, sol)

config.model.input_dim = values.shape[-1]

# Create train and test sets
train_dataset, val_dataset, test_dataset, trajs = generate_train_and_test_sets(values, **config.data.train_test_params)

# Initialize model
model = SequenceModel(config)

data_module = SequenceDataModule(
    datasets=(train_dataset, val_dataset, test_dataset),
    batch_size=config.training.data.batch_size,
    num_workers=config.training.data.num_workers,
    pin_memory=config.training.data.pin_memory,
)
# Train
trainer.fit(model, data_module)

# Test
trainer.test(model, data_module)

# Close wandb run
wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Sequence Indices: 100%|██████████| 1175/1175 [00:00<00:00, 156038.10it/s]
/nfs/home2/eisenaj/miniforge3/envs/reg_emb/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | LRUMinimal | 366 K  | train
1 | loss_fn | MSELoss    | 0      | train
-----------------------------------------------
366 K     Trainable params
0         Non-trainable params
366 K     Total params
1.467     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Train dataset shape: torch.Size([16450, 25, 3])
Validation dataset shape: torch.Size([4700, 25, 3])
Test dataset shape: torch.Size([2350, 25, 3])
Train trajectories dataset shape: torch.Size([14, 1200, 3])
Validation trajectories dataset shape: torch.Size([4, 1200, 3])
Test trajectories dataset shape: torch.Size([2, 1200, 3])
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

In [39]:
batch = next(iter(data_module.train_dataloader()))

In [40]:
batch.shape

torch.Size([32, 25, 3])

In [30]:
model.model

{'params': LRUMinimal(
  (encoder): Linear(in_features=64, out_features=256, bias=True)
  (ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (lru): LRUBlock()
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=1024, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1024, out_features=64, bias=True)
  )
)}